<a href="https://colab.research.google.com/github/SourCherries/VAE-GAN-CelebA/blob/master/make_z_and_decoded_for_comparison_with_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Download Tensorflow checkpoint
! gdown --id 1o9nUhtZEX9GZXsTmvCXSNjEjDQ7Q1wHV

%tensorflow_version 1.x

!git clone https://github.com/SourCherries/VAE-GAN-CelebA.git

sys.path.insert(0,'/content/VAE-GAN-CelebA/libs')
sys.path.insert(0,'/content/VAE-GAN-CelebA/')

file = "/content/VAE-GAN-CelebA/example.jpg"
print('Input file is ', file)

try:
    import os
    import numpy as np
    import tensorflow as tf
    import matplotlib.pyplot as plt
    import libs.make_network_deterministic as make_network
    import libs.utils as utils
    from scipy.io import loadmat, savemat
except ImportError as e:
    print("Make sure the libs folder is available in current directory.")
    print(e)

# Encoding

In [ ]:
file = "example.jpg"

sess, X, G, Z, Z_mu, is_training, saver = make_network.make_network()
if os.path.exists("vaegan_celeba.ckpt"):
    saver.restore(sess, "vaegan_celeba.ckpt")
    print("VAE-GAN model restored.")
else:
    print("Pre-trained network appears to be missing.")
    sys.exit()

img = plt.imread(file)[..., :3]
img = utils.preprocess128(img,crop_factor=0.8)[np.newaxis]
                
#generate images from z
z = sess.run(Z_mu, feed_dict={X: img, is_training: False})

#save data in Matlab format
savemat(file[:-4]+'_z',dict(latent=z))

# Decoding

In [ ]:
file = "example_z"

sess, X, G, Z, Z_mu, is_training, saver = make_network.make_network()
if os.path.exists("vaegan_celeba.ckpt"):
    saver.restore(sess, "vaegan_celeba.ckpt")
    print("VAE-GAN model restored.")
else:
    print("Pre-trained network appears to be missing.")
    sys.exit()


#load data from Matlab format
latent = loadmat(file)['latent'] #make sure that the matlab variable name is 'latent'
print('input data to be transformed:',latent.shape)
if latent.shape[0]==1024:
    latent=latent[np.newaxis]
elif latent.shape[1]!=1024:
    print("None of the input dimensions appears to be 1024!!!")
                
#generate images from z
g = sess.run(G, feed_dict={Z: (1*latent), is_training: False})

def imdeprocess(g):
    stretch = 1.0
    for i in range(g.shape[0]):
        g[i]=np.clip(stretch*g[i] / (g.max()),0,1)
    return g

g=imdeprocess(g)
                
#if there's only one image, we save it as a jpg
if g.shape[0]==1:
    plt.imsave(file[:-4]+'_g.jpg',g[0])
                
#in all cases, we save the image(s) as a .mat file
savemat(file[:-4]+'_g',dict(images=g))